## Queries mais complexas com SQL Alchemy
***

Vamos primeiro realizar a conexão

In [1]:
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import sessionmaker
url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using (psycopg2)
    username='notebook',
    password='notebook',
    host='postgres',
    database='notebook',
    port=5432
)
engine = create_engine(url, pool_size=10, max_overflow=20, pool_recycle=3600)
Session = sessionmaker(bind=engine)

In [2]:
from sqlalchemy import INTEGER
from sqlalchemy.orm import DeclarativeBase


class Base(DeclarativeBase):
    pass

***
### Estabelecendo relacionamentos em tabelas
***

Definir relações no SQLAlchemy ORM é um processo ligeiramente fácil de configurar atributos nas suas classes de mapeamento e estratégias de carregamento nas suas consultas.

Recomendamos que você visite a documentação do SQLAlchemy. Em particular, verifique esta página para aprender sobre diferentes tipos de padrões de relacionamento, como um-para-um, muitos-para-um, um-para-muitos, muitos-para-muitos.

Como este é um ambiente Jupyter Notebook, vamos redefinir nossas classes de mapeamento, mas no mundo real você pode simplesmente adicionar atributos e pronto. Vamos prosseguir.

In [3]:
import datetime
from typing import Optional
from sqlalchemy.dialects.postgresql import TIMESTAMP
from sqlalchemy.sql.functions import func
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey, BIGINT, VARCHAR, String, DECIMAL

class TimestampMixin:
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    updated_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now(), onupdate=func.now())


class User(Base, TimestampMixin):
    """
    Classe de usuário no banco de dados
    """

    __tablename__ = "users"

    telegram_id: Mapped[int] = mapped_column(BIGINT, primary_key=True)
    full_name: Mapped[str] = mapped_column(VARCHAR(255))
    username: Mapped[Optional[str]] = mapped_column(VARCHAR(255), nullable=True)
    language_code: Mapped[str] = mapped_column(VARCHAR(255))
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    referrer_id: Mapped[Optional[int]] = mapped_column(BIGINT, ForeignKey('users.telegram_id', ondelete='SET NULL'))
    orders: Mapped[list['Order']] = relationship(back_populates='user')
    
class Order(Base, TimestampMixin):
    """
    Tabela de pedidos
    """

    __tablename__ = "orders"

    order_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    user_id: Mapped[int] = mapped_column(BIGINT, ForeignKey("users.telegram_id", ondelete="CASCADE"))
    products: Mapped[list['OrderProducts']] = relationship()
    user: Mapped['User'] = relationship(back_populates='orders')
    
class Product(Base, TimestampMixin):
    """
    Tabela de produtos
    """
    
    __tablename__ = "products"

    product_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    title: Mapped[str] = mapped_column(String(255))
    description: Mapped[str]
    price: Mapped[float] = mapped_column(DECIMAL(precision=16, scale=4))

class OrderProducts(Base):
    """
    Cria a tabela estrangeira do relacionamento entre pedido e produtos
    """
    
    __tablename__ = "order-products"

    order_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("orders.order_id", ondelete="CASCADE"), primary_key=True)
    product_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("products.product_id", ondelete="RESTRICT"), primary_key=True)
    quantity: Mapped[int]
    product: Mapped['Product'] = relationship()
    
# Você pode deletar todas as tabelas com esse comando
Base.metadata.drop_all(engine)

# E recria-las com esse comando
Base.metadata.create_all(engine)

In [4]:
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import select


class Repo:
    """
    Repositorio
    """

    def __init__(self, session: Session):
        """
        Construtor.
        """
    
        self.session = session

    def add_user(
        self,
        telegram_id: int,
        full_name: str,
        language_code: str,
        username: str = None,
        referrer_id: int = None,
    ) -> User:
        """
        Adiciona um usuário.
        """

        insert_stmt = insert(
            User,
        ).values(
            telegram_id=telegram_id,
            full_name=full_name,
            language_code=language_code,
            username=username,
            referrer_id=referrer_id,
        ).on_conflict_do_update(
            index_elements=[User.telegram_id],
            set_=dict(
                username=username,
                full_name=full_name,
            ),
        ).returning(User)
        stmt = select(User).from_statement(insert_stmt)
        result = self.session.scalars(stmt)
        self.session.commit()
        return result.first()

    def add_order(self, user_id: int) -> Order:
        """
        Adicionar um pedido
        """

        stmt = select(Order).from_statement(
            insert(Order).values(user_id=user_id).returning(Order),
        )
        result = self.session.scalars(stmt)
        self.session.commit()
        return result.first()

    def add_product(self, title: str, description: str, price: int) -> Product:
        """
        Adiciona um produto.
        """

        stmt = select(Product).from_statement(
            insert(Product)
            .values(title=title, description=description, price=price)
            .returning(Product),
        )
        result = self.session.scalars(stmt)
        self.session.commit()
        return result.first()

    def add_order_product(self, order_id: int, product_id: int, quantity: int):
        """
        Adiciona um produto em um pedido
        """

        stmt = (
            insert(OrderProducts)
            .values(order_id=order_id, product_id=product_id, quantity=quantity)
        )
        self.session.execute(stmt)
        self.session.commit()

In [5]:
import random
from faker import Faker


def seed_fake_data(repo: Repo):
    """
    Aqui podemos definir algo como uma chave aleatória.
    Se passarmos a mesma semente todas as vezes, obteremos
    a mesma sequência de dados aleatórios.
    
    Vamos predefinir as nossas matrizes de entidades falsas
    para as podermos referenciar para criar relações e(ou) para
    dar referrer_id a alguns utilizadores e assim por diante.
    """

    Faker.seed(0)
    fake = Faker()

    users = []
    orders = []
    products = []

    # add users
    for _ in range(10):
        referrer_id = None if not users else users[-1].telegram_id
        user = repo.add_user(
            telegram_id=fake.pyint(),
            full_name=fake.name(),
            language_code=fake.language_code(),
            username=fake.user_name(),
            referrer_id=referrer_id,
        )
        users.append(user)

    # add orders
    for _ in range(10):
        order = repo.add_order(
            user_id=random.choice(users).telegram_id,
        )
        orders.append(order)

    # add products
    for _ in range(10):
        product = repo.add_product(
            title=fake.word(),
            description=fake.sentence(),
            price=fake.pyint(),
        )
        products.append(product)

    # add products to orders
    for order in orders:
        # Here we use `sample` function to get list of 3 unique products
        for product in random.sample(products, 3):
            repo.add_order_product(
                order_id=order.order_id,
                product_id=product.product_id,
                quantity=fake.pyint(),
            )

In [6]:
# Vamos popular nosso banco de dados
with Session() as session:
    repo = Repo(session)
    seed_fake_data(repo)

***
### Vamos começar as queries
***

In [7]:
from sqlalchemy import select

# Vamos realizar uma consulta pelo id
with Session() as session:
    stmt = select(User).where(User.telegram_id == 18)
    result = session.execute(stmt)
    data = result.scalars().first()
    print(data.full_name)

Samantha Cook


In [8]:
from sqlalchemy import select

# Vamos pegar todos os usuários
with Session() as session:
    stmt = select(User)
    result = session.execute(stmt)
    data = result.scalars().all()
    print([user.full_name for user in data])

['Jennifer Green', 'Heather Snow', 'Ms. Michele Guzman', 'Heather Stewart', 'Robert Dunn', 'Samantha Cook', 'Brenda Daniels', 'Juan Mann', 'Donna Davis', 'Amanda Zavala']


In [9]:
from sqlalchemy import select, or_

# Vamos realizar uma query para buscar todos os usuário um pouco mais avançada
with Session() as session:
    stmt = select(
        User,
    ).where(
        # A sintaxe das cláusulas OR é apenas explícita, ao contrário da cláusula AND.
        # É possível passar cada argumento da instrução OR como argumentos para
        # a função `sqlalchemy.or_`, como no exemplo abaixo
        or_(
            User.language_code == 'ro',
            User.language_code == 'uk',
        ),
        # Cada argumento que você passa para o método `where` do objeto Select
        # é considerado como um argumento da instrução AND
        User.username.ilike('%ysu%'),
    ).order_by(
        User.created_at.desc(),
    ).limit(
        10,
    ).having(
        User.telegram_id > 0,
    ).group_by(
        User.telegram_id,
    )
    result = session.execute(stmt)
    data = result.scalars().all()
    print([user.full_name for user in data])

['Jennifer Green']


**!!!IMPORTANTE!!!** Observe a declaração de importação no início desta célula. Os métodos `.returning(...)` e `.on_conflict_do_nothing()` (assim como `.on_conflict_do_update(...)`) não são acessíveis utilizando o construtor básico `sqlalchemy.insert`. Estas são partes do dialeto do PostgreSQL.  O que estamos tentando alcançar? Nós queremos INSERIR o usuário toda vez e SELECIONÁ-LO se nenhum conflito ocorrer (no lado do banco de dados). E, se houver um conflito, fazer UPDATE e só depois um SELECT na linha atualizada.

In [10]:
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import select, or_

# Vamos combinar tudo que vimos em uma única query
with Session() as session:
    insert_stmt = insert(
        User,
    ).values(
        telegram_id=2,
        full_name='Juan Perez',
        language_code='es',
        username='juanpe',
        referrer_id=None,
    # Aqui estamos a utilizar um novo método que representa a instrução
    # RETURNING em SQL puro (particularmente a sintaxe do PostgreSQL)
    ).returning(
        User,
    # Além disso, outro método que utiliza a instrução bruta do PostgreSQL,
    # como ON CONFLICT DO. Nesse caso, estamos utilizando ON CONFLICT DO UPDATE,
    # mas ON CONFLICT DO NOTHING também é possível utilizando o método `.on_conflict_do_nothing()`.
    ).on_conflict_do_update(
        # O argumento `index_elements` refere-se a um conjunto de entidades utilizadas
        # para distinguir os registos uns dos outros
        index_elements=[User.telegram_id],
        # Argumento `set_` (adicionamos sublinhado no final porque `set` é um nome
        # reservado em python, não o podemos utilizar como chave) utilizado para definir
        # quais as colunas que pretende atualizar em caso de conflito. Quase idêntico ao
        # uso do método `.values()`
        set_=dict(
            username='juanpe',
            full_name='Juan Perez',
        ),
    )
    # E aqui estamos a declarar que queremos realizar um SELECT
    # na entidade da nossa declaração INSERT.
    stmt = select(User).from_statement(insert_stmt)
    # Além disso, aqui está outra maneira de executar sua declaração e recuperar dados.
    # Você pode usar `session.scalars(stmt)` em vez de `session.execute(stmt).scalars()`
    result = session.scalars(stmt).first()
    session.commit()
    print([user.full_name for user in data])

['Jennifer Green']


***
### Consultas ORM JOIN (INNER, OUTER)
***

Vamos criar um método para obter todos os usuário convidados (referrer_id IS NOT NULL)

In [11]:
from sqlalchemy.orm import aliased
from sqlalchemy import select

with Session() as session:
    ParentUser = aliased(User)
    ReferralUser = aliased(User)
    
    stmt = (
        select(
            ParentUser.full_name.label('parent_name'),
            ReferralUser.full_name.label('referral_name'),
        ).join(
            ReferralUser, ReferralUser.referrer_id == ParentUser.telegram_id,
        )
    )
    result = session.execute(stmt)
    data = result.all()
    for row in data:
        print(f'Parent: {row.parent_name}, Referral: {row.referral_name}')

Parent: Jennifer Green, Referral: Heather Snow
Parent: Heather Snow, Referral: Ms. Michele Guzman
Parent: Ms. Michele Guzman, Referral: Heather Stewart
Parent: Heather Stewart, Referral: Robert Dunn
Parent: Robert Dunn, Referral: Samantha Cook
Parent: Samantha Cook, Referral: Brenda Daniels
Parent: Brenda Daniels, Referral: Juan Mann
Parent: Juan Mann, Referral: Donna Davis
Parent: Donna Davis, Referral: Amanda Zavala


***
### Consultas Select avançadas com Joins no SQLAlchemy ORM
***

In [12]:
from sqlalchemy import select

with Session() as session:
    stmt = select(User)
    result = session.execute(stmt)
    data = result.scalars().all()
    for user in data:
        print(f'User: {user.full_name} ({user.telegram_id})')
        for order in user.orders:
            print(f'\tOrder: {order.order_id}')
            for product_association in order.products:
                print(f'\t\tProduct: {product_association.product.title}')

User: Jennifer Green (6311)
User: Heather Snow (4969)
User: Ms. Michele Guzman (4104)
	Order: 2
		Product: sometimes
		Product: benefit
		Product: blood
	Order: 3
		Product: develop
		Product: play
		Product: blood
	Order: 9
		Product: play
		Product: go
		Product: develop
User: Heather Stewart (7735)
	Order: 8
		Product: blood
		Product: play
		Product: develop
User: Robert Dunn (8541)
	Order: 5
		Product: available
		Product: particularly
		Product: benefit
	Order: 6
		Product: result
		Product: go
		Product: sometimes
User: Samantha Cook (18)
User: Brenda Daniels (3909)
	Order: 1
		Product: particularly
		Product: develop
		Product: available
	Order: 4
		Product: develop
		Product: result
		Product: particularly
User: Juan Mann (8016)
User: Donna Davis (9882)
User: Amanda Zavala (3102)
	Order: 7
		Product: play
		Product: particularly
		Product: blood
	Order: 10
		Product: sometimes
		Product: benefit
		Product: develop
User: Juan Perez (2)


A partir do resultado da célula acima, podemos ver que funciona, mas executa muitas instruções, pelo que não foram utilizados JOINs. É uma maneira muito ineficaz.

Vamos fazer a declaração real com joins e relacionamentos SQLAlchemy!

In [13]:
from sqlalchemy import select

with Session() as session:
    stmt = select(Order, User).join(User.orders).where(User.telegram_id == 4104)
    result = session.execute(stmt)
    user_orders = result.all()
    
    for order, user in user_orders:
        print(f'Order: {order.order_id} - {user.full_name}')
    print('=============')
    
    for row in user_orders:
        print(f'Order: {row.Order.order_id} - {row.User.full_name}')

Order: 2 - Ms. Michele Guzman
Order: 3 - Ms. Michele Guzman
Order: 9 - Ms. Michele Guzman
Order: 2 - Ms. Michele Guzman
Order: 3 - Ms. Michele Guzman
Order: 9 - Ms. Michele Guzman


In [14]:
from sqlalchemy import select

# Nos dois exemplos seguintes, pode ver como acessar os seus dados quando não especificou apenas tabelas completas
with Session() as session:
    stmt = select(Order, User.full_name).join(User.orders).where(User.telegram_id == 4104)
    result = session.execute(stmt)
    user_orders = result.all()
    
    for order, full_name in user_orders:
        print(f'Order: {order.order_id} - {full_name}')
    print('=============')

    for row in user_orders:
        # Como pode ver, se especificarmos uma coluna em vez de uma tabela completa,
        # podemos acessa-lo diretamente a partir da linha utilizando o nome da coluna
        print(f'Order: {row.Order.order_id} - {row.full_name}')

Order: 2 - Ms. Michele Guzman
Order: 3 - Ms. Michele Guzman
Order: 9 - Ms. Michele Guzman
Order: 2 - Ms. Michele Guzman
Order: 3 - Ms. Michele Guzman
Order: 9 - Ms. Michele Guzman


Como se pode ver, temos apenas uma consulta executada e todas as orders do usuário especificado são retornadas.

Vamos fazer algo mais avançado!

In [15]:
from sqlalchemy import select

with Session() as session:
    stmt = (
        select(Product, Order, User.full_name, OrderProducts.quantity)
        .join(User.orders)
        .join(Order.products)
        .join(Product)
        .where(User.telegram_id == 4104)
    )
    result = session.execute(stmt)
    user_orders1 = result.all()
    
    stmt = (
        select(Product, Order, User.full_name, OrderProducts.quantity)
        .join(OrderProducts)
        .join(Order)
        .join(User)
        .select_from(Product)
        .where(User.telegram_id == 4104)
    )
    result = session.execute(stmt)
    user_orders2 = result.all()
    
    assert user_orders1 == user_orders2
    
    for product, order, full_name, quantity in user_orders1:
        print(f'#{product.product_id} Product: {product.title} (x {quantity}) Order: {order.order_id}: {full_name}')

#1 Product: sometimes (x 5608) Order: 2: Ms. Michele Guzman
#7 Product: benefit (x 8674) Order: 2: Ms. Michele Guzman
#10 Product: blood (x 4107) Order: 2: Ms. Michele Guzman
#8 Product: develop (x 1920) Order: 3: Ms. Michele Guzman
#6 Product: play (x 9777) Order: 3: Ms. Michele Guzman
#10 Product: blood (x 7246) Order: 3: Ms. Michele Guzman
#6 Product: play (x 5503) Order: 9: Ms. Michele Guzman
#5 Product: go (x 748) Order: 9: Ms. Michele Guzman
#8 Product: develop (x 8918) Order: 9: Ms. Michele Guzman


Assim, com apenas uma consulta, podemos acessar dados de várias tabelas de uma forma bastante simples com o SQLAlchemy.

***
### Consultas agregadas usando SQLAlchemy
***

O SQLAlchemy permite-nos utilizar funções SQL de agregação como SUM, COUNT, MIN/MAX/AVG e assim por diante.

Todas as funções de agregação SQL estão acessíveis com o módulo `sqlalchemy.func`

In [16]:
from sqlalchemy import func


class Repo:
    """
    Repositorio
    """

    def __init__(self, session: Session):
        """
        Construtor
        """

        self.session = session

    def get_user_total_number_of_orders(self, telegram_id: int):
        """
        Pega o número total de pedidos de um usuário.
        """

        stmt = select(func.count(Order.order_id)).where(Order.user_id == telegram_id)
        result = self.session.scalar(stmt)
        return result

    def get_total_number_of_orders_by_user(self):
        """
        Pega o valor total de pedidos agrupados por usuários
        """

        stmt = (
            select(func.count(Order.order_id), User.full_name)
            .join(User)
            .group_by(User.full_name)
        )
        result = self.session.execute(stmt)
        return result.all()

    def get_total_number_of_orders_by_user_with_labels(self):
        """
        Pega o valor total de pedidos agrupados por usuários usando labels
        """

        stmt = (
            select(func.count(Order.order_id).label('quantity'), User.full_name.label('name'))
            .join(User)
            .group_by(User.telegram_id)
        )
        result = self.session.execute(stmt)
        return result.all()

    def get_count_of_products_by_user(self):
        stmt = (
            select(func.sum(OrderProducts.quantity).label('quantity'), User.full_name.label('name'))
            .join(Order, Order.order_id == OrderProducts.order_id)
            .join(User)
            .group_by(User.telegram_id)
        )
        result = self.session.execute(stmt)
        return result.all()

    def get_count_of_products_greater_than_x_by_user(self, greater_than: int):
        """
        Pega o total de produtos que tem a quantidade maior que X
        """

        stmt = (
            select(func.sum(OrderProducts.quantity).label('quantity'), User.full_name.label('name'))
            .join(Order, Order.order_id == OrderProducts.order_id)
            .join(User)
            .group_by(User.telegram_id)
            .having(func.sum(OrderProducts.quantity) > greater_than)
        )
        result = self.session.execute(stmt)
        return result.all()


with Session() as session:
    repo = Repo(session)
    user_telegram_id = 3909  # Brenda Daniels

    user_total_number_of_orders = repo.get_user_total_number_of_orders(telegram_id=user_telegram_id)
    print(f'[User: Brenda Daniels] total de pedidos: {user_total_number_of_orders}')
    print('===========')

    for orders_count, full_name in repo.get_total_number_of_orders_by_user():
        print(f'Total de pedidos: {orders_count}, realizados por {full_name}')
    print('===========')

    for row in repo.get_total_number_of_orders_by_user_with_labels():
        print(f'Total de pedidos: {row.quantity}, realizados por {row.name}')
    print('===========')

    for products_count, name in repo.get_count_of_products_by_user():
        print(f'Total de produtos: {products_count}, comprados por {name}')
    print('===========')

    for products_count, name in repo.get_count_of_products_greater_than_x_by_user(20_000):
        print(f'Total de produtos: {products_count}, comprados por {name}')

[User: Brenda Daniels] total de pedidos: 2
Total de pedidos: 3, realizados por Ms. Michele Guzman
Total de pedidos: 2, realizados por Amanda Zavala
Total de pedidos: 2, realizados por Brenda Daniels
Total de pedidos: 2, realizados por Robert Dunn
Total de pedidos: 1, realizados por Heather Stewart
Total de pedidos: 2, realizados por Robert Dunn
Total de pedidos: 3, realizados por Ms. Michele Guzman
Total de pedidos: 2, realizados por Brenda Daniels
Total de pedidos: 2, realizados por Amanda Zavala
Total de pedidos: 1, realizados por Heather Stewart
Total de produtos: 41691, comprados por Robert Dunn
Total de produtos: 52501, comprados por Ms. Michele Guzman
Total de produtos: 17003, comprados por Brenda Daniels
Total de produtos: 26552, comprados por Amanda Zavala
Total de produtos: 9001, comprados por Heather Stewart
Total de produtos: 41691, comprados por Robert Dunn
Total de produtos: 52501, comprados por Ms. Michele Guzman
Total de produtos: 26552, comprados por Amanda Zavala


In [2]:
# Printar uma query SQL criada pela ORM
print(stm.compile(compile_kwargs={"literal_binds": True}))